In [1]:
### Set up GEE authentication and initialize the project
import ee
import geemap
from src import gee_functions as f

ee.Authenticate()
ee.Initialize(project="ee-dw-water")

In [ ]:
### SET ROI and YEAR

# ROI - load from GEE assets
ROI = ee.FeatureCollection("projects/ee-dw-water/assets/isere_HydroSHEDS_lev06")

# YEAR
YEAR = '2020'

In [ ]:
### LOAD DATA

# run functions and load collection
collection = f.get_monthly_water_occurence_yr(YEAR, ROI)

# print output
print(collection.getInfo())

In [ ]:
### DISPLAY RESULTS ON MAP

#  create geemap object
m = geemap.Map()
m.centerObject(ROI, 10)

# #  visualization parameters
water_prob_vis = {
  'min': 0, 'max': 1,
  'palette': ['yellow', 'green', 'darkblue']
}

# add layers
m.add_layer(collection.select('freq_year'), water_prob_vis, 'Yearly Water Occurence')
# m.add_layer(collection.select('0_freq_month'), water_prob_vis, 'January Water Occurence')

# show map
m

In [ ]:
### EXPORT RESULTS

# select either either only the yearly or all the water frequency bands
exported_collection = collection.select(['freq_year'])
# exported_collection = collection

# set name of file
export_name = 'exported_collection'

# download and save obtained image as geotiff-file
task = ee.batch.Export.image.toDrive(
    image=exported_collection,                              # Specify the image to export
    description='Export water occurence raster',                             # Set the task description
    folder='GEE-exports',                                   # Folder name in Google Drive
    fileNamePrefix=export_name,                         # File name
    region=ROI.geometry().bounds().getInfo()['coordinates'], # Specify the export region
    scale=10,                                               # Set the resolution in meters
    crs='EPSG:4326',                                        # Specify the coordinate reference system
    maxPixels=1e13                                          # Specify the maximum number of pixels to export
)
task.start()